In [3]:
!pip install pdfplumber spacy pandas openpyxl > /dev/null
import spacy
spacy.cli.download("en_core_web_sm")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
from google.colab import files
import os

upload_folder = "uploaded_resumes"
os.makedirs(upload_folder, exist_ok=True)

print("Upload your resume PDFs:")
uploaded_files = files.upload()

# Save to folder
for filename, content in uploaded_files.items():
    with open(os.path.join(upload_folder, filename), 'wb') as f:
        f.write(content)


Upload your resume PDFs:


Saving narvekar.pdf to narvekar (1).pdf
Saving rao.pdf to rao (1).pdf
Saving tare.pdf to tare (1).pdf


In [8]:
def extract_info_from_text(text):
    doc = nlp(text)
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    lower_text = text.lower()

    # Name extraction (top 10 lines)
    name_candidates = [
        line for line in lines[:10]
        if not any(char.isdigit() for char in line)
        and '@' not in line
        and 'experience' not in line.lower()
    ]
    name = name_candidates[0] if name_candidates else None

    # Email
    email_match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
    email = email_match.group() if email_match else None

    # Phone number
    phone_match = re.search(r"(\+?\d{1,3})?[\s\-]?\(?\d{3,4}\)?[\s\-]?\d{3,4}[\s\-]?\d{3,4}", text)
    phone = phone_match.group() if phone_match else None

    # Skills
    skills_list = []
    keywords = [
        'python', 'java', 'c++', 'sql', 'excel', 'machine learning', 'deep learning', 'nlp',
        'pandas', 'numpy', 'django', 'flask', 'javascript', 'react', 'node', 'aws', 'azure',
        'html', 'css', 'git', 'linux', 'figma', 'canva', 'r programming', 'scikit-learn', 'openai'
    ]
    for kw in keywords:
        if kw in lower_text:
            skills_list.append(kw)

    # Section extractor helper
    def extract_section(section_title, stop_titles):
        section_lines = []
        capture = False
        for line in lines:
            if section_title in line.lower():
                capture = True
                continue
            if any(stop in line.lower() for stop in stop_titles):
                if capture:
                    break
            if capture:
                section_lines.append(line)
        return "\n".join(section_lines).strip()

    # Extract full sections
    experience = extract_section("experience", ["skills", "projects", "extra-curricular", "education"])
    projects = extract_section("projects", ["extra-curricular", "education", "skills"])
    extracurricular = extract_section("extra-curricular", ["education", "skills", "summary"])

    return {
        "name": name,
        "email": email,
        "phone": phone,
        "skills": ', '.join(sorted(set(skills_list))),
        "experience": experience or None,
        "projects": projects or None,
        "extra_curricular": extracurricular or None
    }


In [9]:
all_data = []

for file_name in os.listdir(upload_folder):
    if file_name.lower().endswith(".pdf"):
        file_path = os.path.join(upload_folder, file_name)
        with pdfplumber.open(file_path) as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text() or ''

        info = extract_info_from_text(text)
        info['file_name'] = file_name
        all_data.append(info)

# Convert to Excel
df = pd.DataFrame(all_data)
excel_file = "parsed_resume_data.xlsx"
df.to_excel(excel_file, index=False)
files.download(excel_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>